In [3]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import sys
import numpy as np
import glob
import torch

sys.path.insert(1,"/home/showalte/research/prob_seq_queries/")
from seq_queries.utils import read_pkl, write_pkl

In [18]:
def tuple_of_tuples_match(tup1, tup2):
    assert len(tup1) == len(tup2),"Layer lengths do not match"
    for t1,t2 in zip(tup1, tup2):
        assert len(t1) == len(t2),f"Tuples of 2 do not match, t1 {t1.shape} | t2 {t2.shape}"
        t11,t12 = t1
        t21,t22 = t2
        assert torch.equal(t11,t21),\
            f"First tuple position shapes do not match: t11 {t11.shape} | t21 {t21.shape}"
        assert torch.equal(t12,t22),\
            f"Second tuple position shapes do not match: t12 {t12.shape} | t21 {t22.shape}"
    print("All good!")
        

In [29]:
def split_2xtups(hidden_states,max_batch_size = 16):
    hidden_states = list(zip(*[
                    zip(*(torch.split(h[0],max_batch_size), torch.split(h[1],max_batch_size)))
                     for h in hidden_states]))
    print(len(hidden_states))
    print(len(hidden_states[0]))
    print(len(hidden_states[0][0]))
    print(hidden_states[0][0][0].shape)
    return hidden_states

def unsplit_2xtups(step_outputs):
    layer_hiddens = []
    for layer_data in zip(*step_outputs):
            layer_data= list(zip(*layer_data))
            layer_hiddens.append(
                (torch.cat(layer_data[0],dim=0).cpu(),
                 torch.cat(layer_data[1],dim=0).cpu())
            )
    return tuple(layer_hiddens)

In [30]:

def make_hidden_state(num_layers, tup_size, tensor_dim):
    tup = []
    for l in range(num_layers):
        i_tup = []
        for t in range(tup_size):
            i_tup.append(torch.randn(*tensor_dim))
        i_tup = tuple(i_tup)
        tup.append(i_tup)
    return tuple(tup)

In [31]:
num_layers = 8
# Samples x heads x seq_len x head_dim)
tensor_dim = (400, 12, 10, 68)
tup_size = 2
h = make_hidden_state(num_layers, tup_size, tensor_dim)

In [32]:
max_batch_sizes = [1,8,9,31,16,128,32]
for max_batch_size in max_batch_sizes:
    tuple_of_tuples_match(
        h,
        unsplit_2xtups(
            split_2xtups(h,
                         max_batch_size), 
        )

    )

400
8
2
torch.Size([1, 12, 10, 68])
All good!
50
8
2
torch.Size([8, 12, 10, 68])
All good!
45
8
2
torch.Size([9, 12, 10, 68])
All good!
13
8
2
torch.Size([31, 12, 10, 68])
All good!
25
8
2
torch.Size([16, 12, 10, 68])
All good!
4
8
2
torch.Size([128, 12, 10, 68])
All good!
13
8
2
torch.Size([32, 12, 10, 68])
All good!
